# 1. 사전 세팅

In [5]:
import warnings

warnings.filterwarnings('ignore')

%cd "/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링"

!gdown --folder "https://drive.google.com/drive/folders/1c6IubbBK9wMVwVDXoD4VHVQOGTBLVoQu?usp=drive_link" --remaining-ok

/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링
Retrieving folder contents
Processing file 1YV_BQoRE7Vg9JZnZ66mBsyj7DXirpphu plot_data.csv
Processing file 128jOEnLa7GZDbDedycK6aKVMrkHSICIM processed.csv
Processing file 1N1r59LgOo0aEe6XrHJD8N7OkYaqh5r1T sales_data.csv
Processing file 1TeUbuEsMiYggQTPOZ5D3bm0Mk-imyFDD social1.csv
Processing file 1cUu-PtqjGZ3AbSO6ZLgSoS0tIrrfSqxT social2.csv
Processing file 1ulu2RcZdI0MU86Gs2ORiVdSHp0KXOSWk social3.csv
Processing file 1DVCJGHMpt6pU3xUOsZZLDFoMu99-83kx test.csv
Processing file 1EzC3fHbeFs1QEwxa5kxDMqkBYlOv8eqD WEAR_TS_ALL.xlsx
Processing file 1n3qi-mThWMA1j6ZzksQREPDzNdNzWSZx weather_2016_2020.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1YV_BQoRE7Vg9JZnZ66mBsyj7DXirpphu
To: /content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링/Data/plot_data.csv
100% 73.8k/73.8k [00:00<00:00, 37.4MB/s]
Dow

In [7]:
!pwd

/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링


# 2. 문제 정의


---

    A 의류 회사에서 생산량을 조절하기 위해 판매량을 예측하려고 한다.
    의류 종류에 따라 계절성을 타기 때문에, 이런 특징을 고려할 수 있는 모델링이 필요하다.



## 도메인 지식

---

티셔츠 제조의 Lead Time(주문부터 납품까지 걸리는 전체 시간)

    티셔츠 발주를 넣어 생산하는 데 통상 2~3주가 걸린다.
    그러나, 보통 베트남이나 중국 등의 해외에서의 생산을 주로 하기 때문에 유통 시간까지 고려해서 계산해야 한다.
    배에 싣거나, 검사하거나, 이동하는 시간까지 고려하게 되면 총 8주 정도가 걸린다고 예상한다.

재고 비용

    재고를 가지고 있을 때 발생하는 비용으로, 재고가 생기면 창고에 적재되어야 하므로 창고 사용료, 운영료 등을 줄이는 것을 목적으로 해야 한다.
    그렇기에 생산량을 조절해 재고를 최소화하는 것이 매우 중요하다.

왜도와 첨도

    왜도(7일 동안의 평균의 왜도)를 확인하면 현재 판매량의 트렌드를 확인할 수 있다.
    첨도가 높게 되면 쏠림 현상이 일어나기 때문에, 특정 이벤트가 있었음을 짐작할 수 있다.

# 3. 데이터 확인

## 데이터 명세

---

    데이터는 종류/날짜별로 한 행씩 구분되어 총 500개 가량의 서로 다른 티셔츠가 제공되어 있다.

| Column | Description |
|:----|:----|
| SEASON | SS(Spring/Summer) or AW(Autumn/Winter) |
| Type | 국내, 해외 등 |
| ... | 날짜 등 |

## 데이터 살펴보기

In [4]:
%cd "/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링"
!ls

/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링
'1. 의류 판매량 예측 모델링.ipynb'   Data


In [1]:
# Apache MXNet: 딥러닝 모델이 실제로 계산되는 엔진
# GluonTS: MXNet 위에서 동작하는 시계열 예측용 라이브러리
%pip install mxnet gluonts --quiet

In [5]:
import pandas as pd

# Default가 max_columns은 20, max_rows는 60이니까 늘려서 안 짤리게 하기
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

df = pd.read_excel('Data/WEAR_TS_ALL.xlsx')
df.head()

,SEASON,TYPE,2016-01-01 00:00:00,2016-01-02 00:00:00,2016-01-03 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,2016-01-10 00:00:00,2016-01-11 00:00:00,2016-01-12 00:00:00,2016-01-13 00:00:00,2016-01-14 00:00:00,2016-01-15 00:00:00,2016-01-16 00:00:00,2016-01-17 00:00:00,2016-01-18 00:00:00,2016-01-19 00:00:00,2016-01-20 00:00:00,2016-01-21 00:00:00,2016-01-22 00:00:00,2016-01-23 00:00:00,2016-01-24 00:00:00,2016-01-25 00:00:00,2016-01-26 00:00:00,2016-01-27 00:00:00,2016-01-28 00:00:00,2016-01-29 00:00:00,2016-01-30 00:00:00,2016-01-31 00:00:00,2016-02-01 00:00:00,2016-02-02 00:00:00,2016-02-03 00:00:00,2016-02-04 00:00:00,2016-02-05 00:00:00,2016-02-06 00:00:00,2016-02-07 00:00:00,2016-02-08 00:00:00,2016-02-09 00:00:00,2016-02-10 00:00:00,2016-02-11 00:00:00,2016-02-12 00:00:00,2016-02-13 00:00:00,2016-02-14 00:00:00,2016-02-15 00:00:00,2016-02-16 00:00:00,2016-02-17 00:00:00,...,2019-11-12 00:00:00,2019-11-13 00:00:00,2019-11-14 00:00:00,2019-11-15 00:00:00,2019-11-16 00:00:00,2019-11-17 00:00:00,2019-11-18 00:00:00,2019-11-19 00:00:00,2019-11-20 00:00:00,2019-11-21 00:00:00,2019-11-22 00:00:00,2019-11-23 00:00:00,2019-11-24 00:00:00,2019-11-25 00:00:00,2019-11-26 00:00:00,2019-11-27 00:00:00,2019-11-28 00:00:00,2019-11-29 00:00:00,2019-11-30 00:00:00,2019-12-01 00:00:00,2019-12-02 00:00:00,2019-12-03 00:00:00,2019-12-04 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-07 00:00:00,2019-12-08 00:00:00,2019-12-09 00:00:00,2019-12-10 00:00:00,2019-12-11 00:00:00,2019-12-12 00:00:00,2019-12-13 00:00:00,2019-12-14 00:00:00,2019-12-15 00:00:00,2019-12-16 00:00:00,2019-12-17 00:00:00,2019-12-18 00:00:00,2019-12-19 00:00:00,2019-12-20 00:00:00,2019-12-21 00:00:00,2019-12-22 00:00:00,2019-12-23 00:00:00,2019-12-24 00:00:00,2019-12-25 00:00:00,2019-12-26 00:00:00,2019-12-27 00:00:00,2019-12-28 00:00:00,2019-12-29 00:00:00,2019-12-30 00:00:00,2019-12-31 00:00:00
0,16SS,국내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16SS,국내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,6.0,4.0,1.0,3.0,5.0,6.0,2.0,9.0,2.0,7.0,4.0,5.0,3.0,19.0,18.0,13.0,3.0,6.0,16.0,19.0,14.0,23.0,3.0,2.0,11.0,13.0,13.0,10.0,18.0,30.0,10.0,13.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16SS,국내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,1.0,2.0,3.0,1.0,3.0,1.0,1.0,6.0,2.0,12.0,14.0,1.0,7.0,3.0,8.0,5.0,11.0,1.0,NaN,11.0,9.0,4.0,5.0,10.0,4.0,5.0,6.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16SS,국내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,7.0,6.0,8.0,4.0,9.0,5.0,1.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16SS,국내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,1.0,2.0,1.0,3.0,4.0,9.0,2.0,NaN,4.0,6.0,3.0,1.0,8.0,1.0,4.0,2.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

    티셔츠마다 판매 시작 일자가 다름을 확인했다.
    각각의 SKU(재고 관리 단위)에 대한 예측을 하게 되면 시간도 오래 걸리고, 데이터의 양도 적어 비효율적이다.
    그렇기에 이들을 따로 보는 것이 아닌 하나로 합쳐 분석할 것이다.

In [7]:
# 515개의 제품군(SKU;Stock Keeping Unit)에 대해 1463개의 행을 보유
df.shape

(515, 1463)

## 데이터 가공 명세

---

| 명세 | 내용 |
|:----|:----|
| 가공 범위 | <input type="checkbox" checked> Feature(Column) 변형<br> <input type="checkbox">기간 단위 데이터 분석 및 Big Volume 특성을 반영하기 위한 데이터 증강<br> <input type="checkbox" checked>다양한 Table을 참조하기 위한 데이터 분할|
| 가공 상세 | <b>Feature(Column) 변형</b><br><br>- 열로 구성되어 있는 날짜 정보를 Transpose하여 행으로 변환<br>- 단변량 데이터에서 정보 취득을 위해(Random Forest 구축을 위해) 각종 통계량으로 변환<br><br><b>데이터 분할</b><br><br>- 검증을 위해 학습 95%, 검증 5%로 분할 |
| 가공 결과 | <b>As is</b><br><br>- row: 515 / column: 1463 / table: 1<br><br> <b>To be(1)</b><br><br>- row: 1461(+946) / column: 1(-1462) / table: 1<br><br> <b>To be(2)</b><br><br>- row: 1308(-153) / column: 15(+14) / table: 1  |